![alt text](../../pythonexposed-high-resolution-logo-black.jpg "Optionele titel")

### Genoemde tuples - Toepassing - Alternatief voor Dictionaries

Eerst een belangrijke waarschuwing: dit werkt eigenlijk alleen voor dictionaries met **string** keys. Keys van dictionaries kunnen andere types data bevatten die gehasht kunnen worden, (inclusief tuples, zolang deze op hun beurt gehashte types bevatten), en deze voorbeelden zullen niet werken met die types woordenboeken.

In [1]:
from collections import namedtuple

In [2]:
data_dict = dict(key1=100, key2=200, key3=300)

In [3]:
Data = namedtuple('Data', data_dict.keys())

In [4]:
Data._fields

('key1', 'key2', 'key3')

Nu kunnen we een instantie van de `Data` named tuple maken met behulp van de data in de `data_dict` dictionary.
We zouden het volgende (slechte idee) kunnen proberen: 

In [15]:
d1 = Data(*data_dict.values())

In [16]:
d1

Data(key1=100, key2=200, key3=300)

Dit lijkt te werken.
Maar bekijk deze tweede dictionary, waar we de keys niet in dezelfde volgorde aanmaken:

In [5]:
data_dict_2 = dict(key1=100, key3=300, key2=200)

In [6]:
d2 = Data(*data_dict_2.values())

In [37]:
d2

Data(key1=100, key2=300, key3=200)

Dit is fout gelopen!
We kunnen niet garanderen dat de volgorde van `values()` dezelfde zal zijn als de volgorde van de keys (in onze named tuple en in de dictionary).

In plaats daarvan moeten we de dictionary zelf unpacken, wat resulteert in keyword argumenten die worden doorgegeven aan de Data constructor:

In [38]:
d2 = Data(**data_dict_2)

In [39]:
d2

Data(key1=100, key2=200, key3=300)

Dus, het patroon om een named tuple te maken vanuit een enkele dictionary is eenvoudig:
Voor elke dictionary d kunnen we een named tuple klasse maken en de gegevens er als volgt in invoegen:

`1. Struct = namedtuple('Struct', d.keys())`

`2. data = Struct(**d)`

Omdat dictionaries nu de volgorde van keys behouden, zal de volgorde van de velden in de named tupel structuur hetzelfde zijn. Als je wilt dat je velden op een andere manier worden gesorteerd, sorteer dan simpelweg de keys wanneer je de genoemde tupelklasse aanmaakt. Bijvoorbeeld, om de keys alfabetisch te laten sorteren kunnen we het volgende doen:

In [7]:
data_dict = dict(first_name='John', last_name='Cleese', age=42, complaint='dead parrot')

In [8]:
data_dict.keys()

dict_keys(['first_name', 'last_name', 'age', 'complaint'])

In [9]:
sorted(data_dict.keys())

['age', 'complaint', 'first_name', 'last_name']

In [10]:
Struct = namedtuple('Struct', sorted(data_dict.keys()))

In [11]:
Struct._fields

('age', 'complaint', 'first_name', 'last_name')

Natuurlijk kunnen we nog steeds de juiste waarden uit de dictionary in de juiste slots in de tuple plaatsen door de dictionary uit te pakken in plaats van alleen de waarden:

In [12]:
d1 = Struct(**data_dict)

In [49]:
d1

Struct(age=42, complaint='dead parrot', first_name='John', last_name='Cleese')

En natuurlijk, aangezien dit nu een named tuple is, kunnen we de gegevens benaderen met de veldnaam:

In [50]:
d1.complaint

'dead parrot'

in plaats van hoe we het met de dictionary zouden hebben gedaan:

In [51]:
data_dict['complaint']

'dead parrot'

Ik wil ook vermelden dat we bij dictionaries vaak code krijgen waarbij de key wordt opgeslagen in een variabele en vervolgens op deze manier wordt aangeroepen:

In [53]:
key_name = 'age'
data_dict[key_name]

42

We kunnen deze aanpak echter niet direct gebruiken met named tuples. Bijvoorbeeld, dit zal niet werken:

In [13]:
key_name = 'age'
d1.key_name

AttributeError: 'Struct' object has no attribute 'key_name'

We kunnen wel de `getattr`-functie gebruiken die we eerder hebben gezien:

In [14]:
key_name = 'age'
getattr(d1, key_name)

42

We hebben ook de `get`-methode op dictionaries die een standaardwaarde kan specificeren om terug te geven als de sleutel niet bestaat:

In [59]:
data_dict.get('age', None), data_dict.get('invalid_key', None)

(42, None)

En we kunnen hetzelfde doen met de `getattr` functie:

In [60]:
getattr(d1, 'age', None), getattr(d1, 'invalid_field', None)

(42, None)

Dit is niet erg nuttig als je alleen werkt met een enkele instantie van een dictionary die dezelfde set keys heeft. 
Je wilt ook niet voor elke instantie van een dictionary een nieuwe named tuple creëren - dat zou gewoon te veel overhead zijn.

Maar in gevallen waar je een verzameling dictionaries hebt die een gemeenschappelijke set keys delen, kan dit echt nuttig zijn, zolang je bereid bent te accepteren dat je nu te maken hebt met onveranderlijke structuren.

Laten we ervan uitgaan dat we deze gegevenslijst hebben:

In [18]:
data_list = [
    {'key1': 1, 'key2': 2},
    {'key1': 3, 'key2': 4},
    {'key1': 5, 'key2': 6, 'key3': 7},
    {'key2': 100}
]

Het eerste wat opvalt is dat we alle mogelijke sleutels moeten achterhalen die zijn gebruikt in de dictionaries in deze lijst.

De makkelijkste manier om dit te doen is door alle sleutels van alle woordenboeken te extraheren en er vervolgens een `set` van te maken, om dubbele sleutelnamen te elimineren:

We zouden het op deze manier kunnen doen, met behulp van een simpele loop:

In [19]:
keys = set()
for d in data_list:
    for key in d.keys():
        keys.add(key)

In [20]:
keys

{'key1', 'key2', 'key3'}

Maar eigenlijk zou een meer efficiënte manier zijn om een ​​comprehensie te gebruiken:

In [21]:
keys = {key for dict_ in data_list for key in dict_.keys()}

In [22]:
keys

{'key1', 'key2', 'key3'}

In feite kunnen we ook het feit gebruiken dat we meerdere sets kunnen verenigen (dit zullen we later behandelen) door alle keys uit te pakken en een union ervan te creëren:

In [23]:
keys = set().union(*(dict_.keys() for dict_ in data_list))

In [24]:
keys

{'key1', 'key2', 'key3'}

Hoe je het ook doet, we eindigen met een verzameling van alle mogelijke keys die gebruikt worden in onze lijst van dictionaries.

Nu kunnen we verder gaan en een named tuple maken met al die keys als velden:

In [25]:
Struct = namedtuple('Struct', keys)

In [26]:
Struct._fields

('key2', 'key1', 'key3')

Zoals je kunt zien, behouden sets geen volgorde, dus in dit geval zullen we waarschijnlijk de keys sorteren om onze named tuple te maken:

In [27]:
Struct = namedtuple('Struct', sorted(keys))

In [120]:
Struct._fields

('key1', 'key2', 'key3')

Nu gaan we ook default values invoeren, aangezien niet alle dictionaries alle keys bevatten. In dit geval ga ik de standaardwaarde instellen op `None` als de sleutel ontbreekt:

In [28]:
Struct.__new__.__defaults__ = (None,) * len(Struct._fields)

Nu zijn we klaar om al deze dictionaries in te laden in een nieuwe lijst van named tuples:

In [29]:
tuple_list = [Struct(**dict_) for dict_ in data_list]

In [123]:
tuple_list

[Struct(key1=1, key2=2, key3=None),
 Struct(key1=3, key2=4, key3=None),
 Struct(key1=5, key2=6, key3=7),
 Struct(key1=None, key2=100, key3=None)]

Tot slot kunnen we dit allemaal netjes verpakken in een enkele functie die een iterable van dictionaries zal aannemen, of een willekeurig aantal dictionaries als positionele argumenten, en een lijst van named tuples zal retourneren:

In [5]:
def tuplify_dicts(dicts):
    keys = {key for dict_ in dicts for key in dict_.keys()}
    Struct = namedtuple('Struct', keys)
    Struct.__new__.__defaults__ = (None,) * len(Struct._fields)
    return [Struct(**dict_) for dict_ in dicts]

In [6]:
tuplify_dicts(data_list)

[Struct(key1=1, key2=2, key3=None),
 Struct(key1=3, key2=4, key3=None),
 Struct(key1=5, key2=6, key3=7),
 Struct(key1=None, key2=100, key3=None)]

Is Python niet leuk? :-)